<a href="https://colab.research.google.com/github/Batinnalp/SQLite-/blob/main/SQLite_subquery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

cursor.execute("Create table ogrenciler (id integer primary key, ad text)")
cursor.execute("Create table notlar (ogrenci_id integer, ders text, notu integer)")

ogrenciler = [(1, "Ali"), (2, "Ayşe"), (3, "Mehmet"), (4, "Zeynep")]
notlar = [
    (1, "Matematik", 85),
    (2, "Matematik", 92),
    (3, "Matematik", 78),
    (4, "Matematik", 88),
    (2, "Fen", 90),
    (3, "Fen", 70)
]

cursor.executemany("insert into ogrenciler (id, ad) values (?, ?)", ogrenciler)
cursor.executemany("insert into notlar (ogrenci_id, ders, notu) values (?, ?, ?)", notlar)
conn.commit()

In [ ]:
cursor.execute("""
select ad
from ogrenciler
where id = (
  select ogrenci_id
  from notlar
  where ders = 'Matematik'
  order by notu desc
  limit 1
)
""")
print(cursor.fetchall())

[('Ayşe',)]


In [ ]:
cursor.execute("""
select ad
from ogrenciler
where id in (
  select ogrenci_id
  from notlar
  where ders = 'Matematik'
  and notu > (
    select avg(notu) from notlar where ders = 'Matematik'
  )
)
""")
print(cursor.fetchall())

[('Ayşe',), ('Zeynep',)]


In [ ]:
cursor.execute("""
select ad, notu
from ogrenciler
join notlar on ogrenciler.id = notlar.ogrenci_id
where notu = (
  select min(notu) from notlar
)
""")
print(cursor.fetchall())

[('Mehmet', 70)]


In [ ]:
cursor.execute("""
select ad, (
  select count(*)
  from notlar
  where ogrenciler.id = notlar.ogrenci_id
) as ders_sayisi
from ogrenciler
""")
print(cursor.fetchall())

[('Ali', 1), ('Ayşe', 2), ('Mehmet', 2), ('Zeynep', 1)]


In [ ]:
cursor.execute("""
select ad
from ogrenciler
where id not in (
  select distinct ogrenci_id from notlar
)
""")
print(cursor.fetchall())

[]
